# A/B Test Analysis for Marketing Campaigns
Simulated example with two variants (A vs B).

In [ ]:

import numpy as np, pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

# Simulate data: impressions, clicks, conversions
rng = np.random.default_rng(123)
nA, nB = 2000, 2000
ctrA, ctrB = 0.065, 0.075
convA, convB = 0.12, 0.135  # conversion rate given a click

impressions = pd.DataFrame({
    'variant': ['A']*nA + ['B']*nB,
    'clicked': np.r_[rng.random(nA) < ctrA, rng.random(nB) < ctrB]
})
clicks = impressions[impressions['clicked']].copy()
clicks['converted'] = (rng.random(len(clicks)) < np.where(clicks['variant']=='A', convA, convB))

# CTR
ctr = impressions.groupby('variant')['clicked'].mean().rename('ctr').reset_index()
# CVR conditional on click
cvr = clicks.groupby('variant')['converted'].mean().rename('cvr').reset_index()

print("CTR\n", ctr)
print("\nCVR (given click)\n", cvr)


## Two-proportion z-test for CTR uplift

In [ ]:

def two_prop_ztest(success_a, total_a, success_b, total_b):
    p1 = success_a / total_a
    p2 = success_b / total_b
    p = (success_a + success_b) / (total_a + total_b)
    se = (p*(1-p)*(1/total_a + 1/total_b))**0.5
    z = (p1 - p2) / se
    pval = 2 * (1 - stats.norm.cdf(abs(z)))
    return z, pval

a_clicks = impressions.query("variant=='A'")['clicked'].sum()
b_clicks = impressions.query("variant=='B'")['clicked'].sum()
z, pval = two_prop_ztest(a_clicks, nA, b_clicks, nB)
print(f"CTR z={z:.3f}, p={pval:.5f}")


## Visualize CTR by variant

In [ ]:

ctr.set_index('variant')['ctr'].plot(kind='bar')
plt.title('CTR by Variant')
plt.ylabel('CTR')
plt.show()


## Notes
- Replace the simulation with your real ad log data.
- Add cost columns to compute CPA/ROI.
- Document business recommendation in the repo README.